In [9]:
import pandas as pd
import numpy as np
from scipy import sparse
import copy
from tqdm import tqdm_notebook

In [10]:
df = pd.read_csv('./interaction_data_ALL_species.csv')
df.columns = ['chembl_id', 'pref_name', 'instance_id', 'canonical_smiles']

In [12]:
print(len(df))

1678621


# Remove duplicates:

In [13]:
df = df.drop_duplicates(['pref_name', 'instance_id'],keep='first').sort_values('pref_name')

In [14]:
keeplist = list()
for i in df['pref_name'].value_counts().items():
    if i[1]>150:
        keeplist.append(i[0])

In [15]:
df = df[df['pref_name'].isin(keeplist)]

In [17]:
len(df)

1011723

In [6]:
df['pref_name'].unique().shape

(851,)

# Make into a n,m matrix:

In [7]:
num_instances = df['instance_id'].unique().shape[0]
num_targets = df['pref_name'].unique().shape[0]

In [8]:
num_instances

630482

In [94]:
interaction_matrix = np.zeros([num_instances, num_targets])


In [95]:
##Setting up containers:
tids = df.sort_values('pref_name')['pref_name'].unique()
cids = df.sort_values('instance_id')['instance_id'].unique()
target_indices = dict()
for count, i in enumerate(tids):
    target_indices[i]=count
    
instance_indices = dict()
for count, i in enumerate(cids):
    instance_indices[i]=count

    
##Filling the values:
for count, item in tqdm_notebook(df.iterrows()):
    t_id = item['pref_name']
    i_id = item['instance_id']
    
    row = instance_indices[i_id]
    column = target_indices[t_id]
    
    interaction_matrix[row, column]=1


In [96]:
for _ in range(100):
    row = np.random.choice(interaction_matrix.shape[0]) #select a random instance
    col = np.random.choice(interaction_matrix[row].nonzero()[0]) #select a label for that instance
    assert tids[col] in list(df[df['instance_id']==cids[row]]['pref_name'])
    
print('passed')
print(interaction_matrix.shape)

passed
(630482, 851)


In [97]:
sparse.save_npz('./interaction_matrix_ALL_species.npz', sparse.csr_matrix(interaction_matrix))

df.sort_values('instance_id').drop_duplicates(['instance_id'])[['instance_id', 'canonical_smiles']].to_csv('all_chemicals')
df.sort_values('pref_name').drop_duplicates(['pref_name'])['pref_name'].to_csv('all_targets', index=False, header=None)





/home/lewis/miniconda3/envs/lew_conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.


In [98]:
pd.read_csv('all_targets', header=None)

,0
0,11-beta-hydroxysteroid dehydrogenase 1
1,15-hydroxyprostaglandin dehydrogenase [NAD+]
2,2-acylglycerol O-acyltransferase 2
3,3-phosphoinositide dependent protein kinase-1
4,5-lipoxygenase activating protein
5,"6-phosphofructo-2-kinase/fructose-2,6-bisphosp..."
6,ADAM17
7,ADAMTS4
8,ADAMTS5
9,ALK tyrosine kinase receptor
